# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
data = pd.read_csv('homepage_actions.csv')

In [3]:
data.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
data.sample(15)

,timestamp,id,group,action
7397,2017-01-07 14:06:57.430190,828946,control,view
4474,2016-11-27 23:08:50.540538,793192,control,view
870,2016-10-06 22:19:11.671858,628217,control,view
2986,2016-11-06 17:42:12.615837,494646,control,view
7762,2017-01-12 13:18:04.825059,895066,experiment,view
6704,2016-12-29 10:09:59.299492,579535,control,view
1316,2016-10-13 05:19:31.561369,929844,control,view
3579,2016-11-15 10:20:31.188069,495048,control,view
3589,2016-11-15 12:38:32.549734,779434,control,click
628,2016-10-03 09:23:54.676360,253531,control,view


In [5]:
data.shape

(8188, 4)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [7]:
#convert timestamp to datetime data type
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [8]:
#check for missing values
missing_values = data.isnull().sum()
print("Missing Values:")
print(missing_values)

Missing Values:
timestamp    0
id           0
group        0
action       0
dtype: int64


In [9]:
# Number of views in the control group
control_views = len(data[(data['group'] == 'control') & (data['action'] == 'view')])

print(control_views)

3332


In [10]:
# Number of views in the experiment group
experiment_views = len(data[(data['group'] == 'experiment') & (data['action'] == 'view')])

print(experiment_views)

2996


In [11]:
# Number of website clicks
clicks = len(data[data['action'] == 'click'])

print(clicks)

1860


In [12]:
#Investigate the 'id' column to answer the provided questions
#How many viewers also clicked?

# Count viewers and click events
viewers_count = (data['action'] == 'view').sum()
click_count = (data['action'] == 'click').sum()

# Calculate the number of viewers who also clicked
viewers_clicked = data[data['action'] == 'click']['id'].nunique()
print(f"Number of viewers: {viewers_count}")
print(f"Number of click events: {click_count}")
print(f"Number of viewers who also clicked: {viewers_clicked}")

Number of viewers: 6328
Number of click events: 1860
Number of viewers who also clicked: 1860


In [13]:
#Are there any anomalies with the data; did anyone click who didn't view?
# Find IDs of users who clicked without viewing
# Create Boolean masks for 'click' and 'view' actions
click_mask = data['action'] == 'click'
view_mask = data['action'] == 'view'

# Get unique IDs for users who clicked without viewing
clicked_without_view = data[click_mask & ~data['id'].isin(data[view_mask]['id'])]['id'].unique()

# Count the number of users who clicked without viewing
num_clicked_without_view = len(clicked_without_view)

print(f"Number of users who clicked without viewing: {num_clicked_without_view}")

Number of users who clicked without viewing: 0


In [14]:
#Is there any overlap between the control and experiment groups?
# Check for overlap between control and experiment groups
control_group_ids = data[data['group'] == 'control']['id']
experiment_group_ids = data[data['group'] == 'experiment']['id']
overlap = control_group_ids.isin(experiment_group_ids).sum()
print(f"Number of overlapping IDs between control and experiment groups: {overlap}")

Number of overlapping IDs between control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [15]:
#The null and alternative hypotheses
H_O = "The experimental homepage and control homepage have the same conversion rate "
H_1 = "The experimental homepage is more effective than the control homepage"

# Calculate conversion rates
experimental_clicks = len(data[(data['group'] == 'experiment') & (data['action'] == 'click')])
control_clicks = len(data[(data['group'] == 'control') & (data['action'] == 'click')])

experimental_views = len(data[(data['group'] == 'experiment') & (data['action'] == 'view')])
control_views = len(data[(data['group'] == 'control') & (data['action'] == 'view')])

p1 = experimental_clicks / experimental_views
p2 = control_clicks / control_views

# Calculate standard error
SE = np.sqrt((p1 * (1 - p1) / experimental_views) + (p2 * (1 - p2) / control_views))

# Calculate the z-score
z = (p1 - p2) / SE

# Set the significance level
alpha = 0.05

# Calculate the critical z-value
critical_z = stats.norm.ppf(1 - alpha)
critical_z

1.6448536269514722

In [16]:
# Compare z-score with critical z-value
if z > critical_z:
    print("Reject the null hypothesis. The experimental homepage is more effective.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference in effectiveness.")

Reject the null hypothesis. The experimental homepage is more effective.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [17]:
#calculate the click rates
control_group = data[data['group'] == 'control']
experiment_group = data[data['group'] == 'experiment']

# Calculate the click-through rate (CTR) for the control group
control_ctr = len(control_group[control_group['action'] == 'click']) / len(control_group[control_group['action'] == 'view'])

# Estimate the expected number of clicks for the experiment group using the control CTR
expected_clicks_experiment = control_ctr * len(experiment_group[experiment_group['action'] == 'view'])

print(expected_clicks_experiment)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [18]:
# Calculate the actual number of clicks in the experiment group
actual_clicks_experiment = len(experiment_group[experiment_group['action'] == 'click'])

# Calculate the z-score
z_score = (actual_clicks_experiment - expected_clicks_experiment) / np.sqrt(expected_clicks_experiment)

print(z_score)

3.1083866623746554


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [19]:
# Calculate the p-value using the normal distribution
p_value = 1 - stats.norm.cdf(z_score)

print(p_value)

0.0009405589049594765


In [20]:
print("""The first set of results has a higher z-score and a lower p-value compared to the second set of results. 
This suggests that the first set of results provides stronger evidence against the null hypothesis and indicates a more significant difference between the experiment and control groups in terms of click-through rates.
In both cases, the results suggest that there is a statistically significant difference between the experiment and control groups.
""")

The first set of results has a higher z-score and a lower p-value compared to the second set of results. 
This suggests that the first set of results provides stronger evidence against the null hypothesis and indicates a more significant difference between the experiment and control groups in terms of click-through rates.
In both cases, the results suggest that there is a statistically significant difference between the experiment and control groups.



### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.